In [2]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [6]:
epochs = 1000
batch_size = 128
patience = 200
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [4]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Resizing(25, 25),
            layers.Rescaling(scale=1./127.5, offset=-1),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(1024,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 25, 25, 3)         0         
                                                                 
 rescaling (Rescaling)       (None, 25, 25, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 23, 23, 256)       7168      
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 256)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 256)         590080    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 4, 4, 256)         0         
 g2D)                                                 

In [5]:
train = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

Found 61938 files belonging to 62 classes.
Using 49551 files for training.
Found 61938 files belonging to 62 classes.
Using 12387 files for validation.


In [7]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/1000


388/388 [==============================] - 70s 177ms/step - loss: 0.0999 - accuracy: 0.9660 - val_loss: 0.0426 - val_accuracy: 0.9847
Epoch 2/1000
388/388 [==============================] - 69s 178ms/step - loss: 0.0893 - accuracy: 0.9685 - val_loss: 0.0386 - val_accuracy: 0.9841
Epoch 3/1000
388/388 [==============================] - 70s 180ms/step - loss: 0.1032 - accuracy: 0.9652 - val_loss: 0.0392 - val_accuracy: 0.9851
Epoch 4/1000
388/388 [==============================] - 69s 176ms/step - loss: 0.1017 - accuracy: 0.9657 - val_loss: 0.0392 - val_accuracy: 0.9849
Epoch 5/1000
388/388 [==============================] - 69s 178ms/step - loss: 0.0981 - accuracy: 0.9670 - val_loss: 0.0401 - val_accuracy: 0.9864
Epoch 6/1000
388/388 [==============================] - 72s 184ms/step - loss: 0.0937 - accuracy: 0.9676 - val_loss: 0.0383 - val_accuracy: 0.9860
Epoch 7/1000
388/388 [==============================] - 72s 184ms/step - loss: 0.0898 - accuracy: 0.9694 - val_loss: